En el caso de LST como proviene de Landsat el primer paso es resamplear a la resolucion de Sentinel 

In [ ]:
import rasterio
from rasterio.enums import Resampling
import os
import glob
from tqdm import tqdm 
import numpy as np
import georasters as gr
import numpy as np
import os
import rasterio
from rasterio.plot import show
from osgeo import gdal

#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide


parque = 'Teide'

# Get the current directory of the script
base_path = os.getcwd()


# Relative paths
input_folder_relative = f"{parque}/LST"
output_folder = f"C:/Users/carlo/Desktop/EFAs/{parque}/reprojected"

## la capa base hay que revisarla por el nombre que tiene 
base_raster_path = f"C:/Users/carlo/Desktop/EFAs/BaseLayers/{parque}/NDWI_SR_SENTINEL_sSD__2018-2022_Season1-12_InterAnnualMedian_Pixel10.tif"
input_files = glob.glob(os.path.join(base_path, input_folder_relative, '*.tif'))


with rasterio.open(base_raster_path) as base_src:
    for input_file in tqdm(input_files, desc="Aligning rasters"):
        with rasterio.open(input_file) as src:
            data = src.read(1)
            profile = src.profile

            # Reproject/resample the input raster to match the base raster
            data_resampled = np.zeros_like(base_src.read(1))
            rasterio.warp.reproject(
                source=data,
                destination=data_resampled,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=base_src.transform,
                dst_crs=base_src.crs,
                resampling=Resampling.nearest
            )

            # Update the profile of the input raster to match the base raster's metadata
            profile_resampled = src.profile.copy()
            profile_resampled.update({
                'transform': base_src.transform,
                'height': data_resampled.shape[0],
                'width': data_resampled.shape[1],
                'crs': base_src.crs
            })
            input_filename = os.path.basename(input_file)

            # Save
            output_path = os.path.join(output_folder, input_filename)
            with rasterio.open(output_path, 'w', **profile_resampled) as dst:
                dst.write(data_resampled, 1)